In [1]:
#import GYM stuff
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 

#import helpers
import numpy as np
import random
import os

#import Stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

#import cnn stuff
import cv2

#import pygame
import pygame
import time
from datetime import datetime

pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.quit()

## CNN 모델 : 코랩 사용 불가 - 비디오카드가 있는 로컬환경에서만 가능

In [6]:
pygame.init()

add="C:/Users/myhome/Documents/my_project/py37_RL/planegame/data/"
black = (0, 0, 0)


class obj:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.move = 0
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img, (sx, sy))
        self.sx, self.sy = self.img.get_size()
    def show(self):
        screen.blit(self.img, (self.x, self.y)) 


# class ProcessFrame84(gym.ObservationWrapper):
    
#     def __init__(self, env=None):
#         super(ProcessFrame84, self).__init__(env)
#         self.observation_space = Box(low=0, high=255, shape=(50, 90, 1), dtype=np.uint8)
    
#     def _observation(self, obs):
        
#         return ProcessFrame84.process(obs)
    
#     @staticmethod
def process1(img):
    img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
    x_t = cv2.resize(img, (40, 80), interpolation=cv2.INTER_AREA)
    x_t = np.reshape(x_t, (40, 80, 1))
    x_t = np.nan_to_num(x_t)

    return x_t.astype(np.uint8)

def crash(a, b):
    if (a.x-b.sx <= b.x) and (b.x <= a.x+a.sx):
        if (a.y-b.sy <= b.y) and (b.y <= a.y+a.sy):
            return True
        else:
            return False
    else:
        return False


class PlaneEnv(Env):
    
    def __init__(self):
        self.action_space = Discrete(3) 
        self.observation_space = Box(low=0, high=255, shape=(40, 80, 1), dtype=np.uint8)
        self.state = np.zeros((40, 80, 1), dtype=np.uint8)
        self.ss = obj()
        self.ss.put_img("{}plane.png".format(add))
        self.ss.change_size(50, 60)
        self.ss.x = 205 #round(size[0]/2 - self.ss.sx/2)
        self.ss.y = 825 #size[1] -self.ss.sy - 15
        self.ss.move = 20
        
    def step(self, action):
        global done
        global reward
        global gamma
        global k
        
        clock.tick(30)
        
        #프레임당 생존 시 리워드 1점 지급
        reward =1
        done=False
        
        full_scr= pygame.surfarray.pixels3d(screen)
        self.state = process1(full_scr)
       
        if action ==0:
            self.ss.x -= self.ss.move
            if self.ss.x <= 0:
                self.ss.x = 0
        elif action ==1:
            self.ss.x += self.ss.move
            if self.ss.x >= 450:
                self.ss.x = 450
        elif action ==2:
            pass
        

        #적군 생성
        rdn = random.random()
        rdn = rdn * gamma


        if rdn > 0.94:
            gamma *= 0.95
            k=1
            aa = obj()
            aa.put_img("{}enemy2.png".format(add))
            aa.change_size(40, 40)
            mylist = [self.ss.x, random.randrange(0,460)]
            aa.x = random.choices(mylist, weights = [3, 10], k = 1)[0]
            aa.y = 10
            aa.move = 25
            a_list.append(aa)
        elif k % 10 == 0 :
            gamma = 1
            

        #적군 삭제 (화면밖으로 벗어날 시)
        d_list = []
        for i in range(len(a_list)):
            ea = a_list[i]
            ea.y += ea.move
            if ea.y >= size[1]:
                d_list.append(i)
                
        for d in d_list:
            del a_list[d]
          
        #에피소드 종료 로직 (충돌 판정)
        for i in range(len(a_list)):
            a = a_list[i]
            if crash(a, self.ss) == True:
                done=True
                reward = -100

        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        global a_list
        # 4-4. 그리기
        screen.fill(black)
        self.ss.show()

        for ea in a_list:
            ea.show()
            

        # 4-5. 업데이트
        pygame.display.flip()

    
    def reset(self):
        self.state = self.state = np.zeros((40, 80, 1), dtype=np.uint8)
        self.ss.x = 205 #round(size[0]/2 - self.ss.sx/2)
        self.ss.y = 825 #size[1] -self.ss.sy - 15
        
        return self.state

In [7]:
# 2. 게임창 옵션 설정
size = [500, 900]
screen = pygame.display.set_mode(size)

title = "My Game"
pygame.display.set_caption(title)

# 3. 게임 내 필요한 설정
clock = pygame.time.Clock()

env=PlaneEnv()


In [8]:
from stable_baselines3.common.env_checker import check_env

state = env.reset()
done = False
score = 0
reward = 0
gamma = 1
k=0
a_list=[]
d_list=[]

check_env(env, warn=True)

In [6]:
ppo_path = os.path.join('Training', 'Saved Models', 'CNN_PPO_2M')
model = PPO.load(ppo_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [7]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    reward = 0
    a_list=[]
    d_list=[]
    
    while not done:
        env.render()
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
#         print(obs, action, score)
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-23
Episode:2 Score:-20
Episode:3 Score:-68
Episode:4 Score:7
Episode:5 Score:5


In [ ]:
pygame.quit()

In [9]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    reward = 0
    a_list=[]
    d_list=[]
    
    while not done:
        k +=1
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
        print(action, score, done)
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

0 1 False
2 2 False
0 3 False
2 4 False
0 5 False
0 6 False
1 7 False
2 8 False
0 9 False
2 10 False
0 11 False
2 12 False
1 13 False
1 14 False
2 15 False
1 16 False
0 17 False
2 18 False
0 19 False
0 20 False
1 21 False
0 22 False
0 23 False
1 24 False
2 25 False
2 26 False
1 27 False
2 28 False
2 29 False
1 30 False
2 31 False
2 32 False
1 33 False
0 34 False
2 35 False
2 36 False
0 37 False
1 38 False
1 39 False
1 40 False
0 41 False
2 42 False
1 43 False
2 44 False
2 45 False
2 46 False
0 47 False
1 48 False
0 49 False
0 50 False
1 51 False
0 52 False
0 53 False
2 54 False
1 55 False
2 56 False
1 57 False
1 58 False
0 59 False
1 60 False
1 61 False
1 62 False
0 63 False
2 64 False
1 65 False
2 66 False
2 67 False
2 68 False
1 69 False
1 70 False
2 71 False
0 72 False
0 73 False
2 74 False
1 75 False
0 76 False
1 77 False
2 78 False
0 79 False
2 80 False
1 81 False
0 82 False
2 83 False
0 84 False
2 85 False
0 86 False
2 87 False
1 88 False
0 89 False
2 90 False
1 91 False
2 92 Fal

In [10]:
pygame.quit()

## Learning

In [2]:
add="C:/Users/myhome/Documents/my_project/py37_RL/planegame/data/"
black = (0, 0, 0)

class obj:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.move = 0
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img, (sx, sy))
        self.sx, self.sy = self.img.get_size()
    def show(self):
        screen.blit(self.img, (self.x, self.y)) 

def process1(img):
    img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
    x_t = cv2.resize(img, (40, 80), interpolation=cv2.INTER_AREA)
    x_t = np.reshape(x_t, (40, 80, 1))
    x_t = np.nan_to_num(x_t)

    return x_t.astype(np.uint8)

def crash(a, b):
    if (a.x-b.sx <= b.x) and (b.x <= a.x+a.sx):
        if (a.y-b.sy <= b.y) and (b.y <= a.y+a.sy):
            return True
        else:
            return False
    else:
        return False


class PlaneEnv(Env):
    
    def __init__(self):
        self.action_space = Discrete(3) 
        self.observation_space = Box(low=0, high=255, shape=(40, 80, 1), dtype=np.uint8)
        self.state = np.zeros((40, 80, 1), dtype=np.uint8)
        self.ss = obj()
        self.ss.put_img("{}plane.png".format(add))
        self.ss.change_size(50, 60)
        self.ss.x = 205 #round(size[0]/2 - self.ss.sx/2)
        self.ss.y = 825 #size[1] -self.ss.sy - 15
        self.ss.move = 20
        
    def step(self, action):
        global done
        global reward
        global gamma
        global k
       
        #프레임당 생존 시 리워드 1점 지급
        reward =1
        done=False
        
        full_scr= pygame.surfarray.pixels3d(screen)
        self.state = process1(full_scr)
       
        if action ==0:
            self.ss.x -= self.ss.move
            if self.ss.x <= 0:
                self.ss.x = 0
        elif action ==1:
            self.ss.x += self.ss.move
            if self.ss.x >= 450:
                self.ss.x = 450
        elif action ==2:
            pass
        

        #적군 생성
        if rdn > 0.94:
            gamma *= 0.95
            k=1
            aa = obj()
            aa.put_img("{}enemy2.png".format(add))
            aa.change_size(40, 40)
            mylist = [self.ss.x, random.randrange(0,460)]
            aa.x = random.choices(mylist, weights = [3, 10], k = 1)[0]
            aa.y = 10
            aa.move = 25
            a_list.append(aa)
        elif k % 10 == 0 :
            print(k)
            gamma = 1
            

        #적군 삭제 (화면밖으로 벗어날 시)
        d_list = []
        for i in range(len(a_list)):
            ea = a_list[i]
            ea.y += ea.move
            if ea.y >= size[1]:
                d_list.append(i)
                
        for d in d_list:
            del a_list[d]
          
        #에피소드 종료 로직 (충돌 판정)
        for i in range(len(a_list)):
            a = a_list[i]
            if crash(a, self.ss) == True:
                done=True
                reward = -100

        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        global a_list
        # 4-4. 그리기
        screen.fill(black)
        self.ss.show()

        for ea in a_list:
            ea.show()
            

        # 4-5. 업데이트
        pygame.display.flip()

    
    def reset(self):
        self.state = self.state = np.zeros((40, 80, 1), dtype=np.uint8)
        self.ss.x = 205 #round(size[0]/2 - self.ss.sx/2)
        self.ss.y = 825 #size[1] -self.ss.sy - 15
        
        return self.state

In [3]:
# 2. 게임창 옵션 설정
size = [500, 900]
screen = pygame.display.set_mode(size)

title = "My Game"
pygame.display.set_caption(title)

env=PlaneEnv()

state = env.reset()
done = False
score = 0
reward = 0
a_list=[]
d_list=[]

In [4]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [5]:
log_path = os.path.join('Training', 'Logs')
# save_path = os.path.join('Training', 'Saved Models')

In [6]:
# stop_callback = StopTrainingOnRewardThreshold(reward_threshold=2000, verbose=1)
# eval_callback = EvalCallback(env, 
#                              callback_on_new_best=stop_callback, 
#                              eval_freq=10000, 
#                              best_model_save_path=save_path, 
#                              verbose=1)

In [8]:
model = PPO("CnnPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [9]:
model.learn(total_timesteps=2000000)#, callback=eval_callback)

Logging to Training\Logs\PPO_34
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.8     |
|    ep_rew_mean     | -79.2    |
| time/              |          |
|    fps             | 132      |
|    iterations      | 1        |
|    time_elapsed    | 15       |
|    total_timesteps | 2048     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 28.2          |
|    ep_rew_mean          | -72.8         |
| time/                   |               |
|    fps                  | 117           |
|    iterations           | 2             |
|    time_elapsed         | 34            |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 1.0894699e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.1          |
|    explain

In [10]:
# plane_path = os.path.join('Training', 'Saved Models', 'evlution_100k_PPO')
plane_path = os.path.join('Training', 'Saved Models', 'CNN_PPO_2M')

model.save(plane_path)

# evaluate_policy(model, env, n_eval_episodes=10, render=False)

In [13]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(-82.9, 43.608370756083055)

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_30')

In [ ]:
!tensorboard --logdir ./Training/Logs/PPO_30